In [ ]:
import os
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import progressbar
import time
import datetime
import re

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
plt.style.use('fivethirtyeight')
%matplotlib inline

# Import

In [ ]:
phishlist = pd.read_csv('d/goverCERTphishes.csv')

# Dates

In [ ]:
phishlist.index = pd.to_datetime(phishlist['Date'], format='%b %d %Y %I:%M %p')

In [ ]:
phishlist.resample('M')['ID'].count().plot()

Looking into beginning of the list

In [ ]:
phishlist.resample('M')['ID'].count().head()

And the end

In [ ]:
phishlist.resample('M')['ID'].count().tail()

Total

In [ ]:
phishlist['URL'].count()

Total Valid Phish

In [ ]:
phishlist['Valid Phish'].value_counts()

In [ ]:
labels = ['Unbekannt', 'Bestätigt','Kein Phishing'] 
phishlist['Valid Phish'].value_counts().plot(kind='pie', figsize=(4,4), autopct='%.1f', labels=labels)

Wrongly files sites by GovCert

In [ ]:
phishlist[phishlist['Valid Phish']=='INVALID']

# Banking
Looking into affected banks

In [ ]:
#Looking into URLs
phishlist['URL'].value_counts().head()

In [ ]:
#Looking for ubs
def small(elem):
    elem = elem.lower()
    return elem

In [ ]:
phishlist['URL'] = phishlist['URL'].apply(small)

In [ ]:
phishlist['ubs'] = phishlist['URL'].str.contains('ubs')
ubs = phishlist[phishlist['ubs']==True]
ubs.resample('M')['ID'].count().plot()

In [ ]:
pd.DataFrame(ubs.resample('M')['ID'].count())

In [ ]:
ubs.head()

In [ ]:
ubs['URL'].count()

In [ ]:
ubs

In [ ]:
phishlist['URL'].count()

Percentage of Phishes

In [ ]:
print("Anteil der UBS phishing-Sites ist hoch:", round(ubs['URL'].count() / phishlist['URL'].count() * 100 , 1), "%")

In [ ]:
ubs[ubs['Valid Phish']=='VALID PHISH']['URL'].count()

In [ ]:
ubs[ubs['Valid Phish']=='VALID PHISH'].head()

In [ ]:
ubs[ubs['Valid Phish']=='Unknown'].head()

In [ ]:
ubs.to_csv('ubs.csv')

# Telecom

In [ ]:
phishlist['sunrise'] = phishlist['URL'].str.contains('sunrise')
sunrise = phishlist[phishlist['sunrise']==True]
sunrise.resample('Q')['ID'].count().plot()

In [ ]:
pd.DataFrame(sunrise.resample('Q')['ID'].count())

In [ ]:
sunrise['sunrise'].count()

In [ ]:
sunrise

In [ ]:
sunrise.to_csv('sunrise.csv')

In [ ]:
phishlist['swisscom'] = phishlist['URL'].str.contains('swisscom')
swisscom = phishlist[phishlist['swisscom']==True]
swisscom.resample('Q')['ID'].count().plot()

In [ ]:
swisscom['swisscom'].count()

# Finding Top Level Domains

In [ ]:
def toplevel(elem):
    elem = elem.replace('https://', '').replace('http://', '')
    elem = elem.split("/")[0].replace('www.','').strip()
    return elem

In [ ]:
def countrydomain(elem):
    elem = elem.replace('https://', '').replace('http://', '')
    elem = elem.split("/")[0].replace('www.','').strip()
    elem = elem.split(".")[-1]
    elem = "."+elem
    return elem

In [ ]:
phishlist['countryleveldomain'] = phishlist['URL'].apply(countrydomain)

In [ ]:
phishlist['countryleveldomain'].value_counts().head(5)

In [ ]:
def realtoplevel(elem):
    if (elem.split('.')[-2] == 'co' or 
        elem.split('.')[-2] == 'com' or
        elem.split('.')[-2] == 'pe' or
        elem.split('.')[-2] == 'org' or
        elem.split('.')[-2] == 'edu' or
        elem.split('.')[-2] == 'ac' or
        elem.split('.')[-2] == 'gov'):
            minilst = elem.split('.')[-4:-1]
            elem = ''
            for e in minilst:
                e = e + "."
                elem += e 
            elem = elem[:-1]
    else:
        elem = elem.split('.')[-2]
    return elem

In [ ]:
phishlist['temp'] = phishlist['URL'].apply(toplevel).apply(realtoplevel)

In [ ]:
phishlist['finaltopleveldoms'] = phishlist['temp'] + phishlist['countryleveldomain']

In [ ]:
phishlist['finaltopleveldoms'].value_counts().head()

In [ ]:
# How many are Swiss
phishlist[phishlist['countryleveldomain']=='.ch']['temp'].value_counts().head()

In [ ]:
phishlist[phishlist['countryleveldomain']=='.ch']

In [ ]:
pd.set_option(".max_colwidth",-1)

In [ ]:
phishlist[phishlist['countryleveldomain']=='.ch']

In [ ]:
phishlist[phishlist['Valid Phish']=='INVALID']

In [ ]:
phishlist.resample('D')['URL'].count().mean()